In [ ]:
import os
import argparse
import logging
logging.basicConfig(level=logging.DEBUG)
from common import find_mxnet, data, fit
from common.util import download_file
from ilab_iterator import ilab_iterator, Multi_ilab_iterator, Single_ilab_iterator, random_task_iterator
import mxnet as mx
import numpy as np
import numpy 
from mxnet import nd


In [ ]:
class Cross_Entropy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self):
        super(Cross_Entropy, self).__init__('cross-entropy')
    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)
        label = labels[0].asnumpy()
        pred = preds[0].asnumpy()
        for i in range(label.shape[0]):
            prob = pred[i,numpy.int64(label[i])]
            if len(labels) == 1:
                self.sum_metric += (-numpy.log(prob)).sum()
        self.num_inst += label.shape[0]
        

def imagenet_iterator(data_dir,batch_size):
    train = mx.io.ImageRecordIter(
            path_imgrec         = os.path.join(data_dir, "imagenet_small_train.rec"),
            label_width         = 1,
            data_name           = 'data',
            label_name          = 'softmax1_label',
            data_shape          = (3, 224, 224),
            batch_size          = batch_size,
            pad                 = 0,
            fill_value          = 127,  # only used when pad is valid
            rand_crop           = True,
            max_random_scale    = 1.0,  # 480 with imagnet, 32 with cifar10
            min_random_scale    = 0.533,  # 256.0/480.0
            max_aspect_ratio    =  0.25,
            random_h            = 36,  # 0.4*90
            random_s            = 50,  # 0.4*127
            random_l            = 50,  # 0.4*127
            #max_rotate_angle    = 10,
            #max_shear_ratio     = 0.1, #
            rand_mirror         = True,
            shuffle             = True)
            #num_parts           = kv.num_workers,
            #part_index          = kv.rank)
    val = mx.io.ImageRecordIter(
            path_imgrec         = os.path.join(data_dir, "imagenet_small_val.rec"),
            label_width         = 1,
            data_name           = 'data',
            label_name          = 'softmax1_label',
            batch_size          = batch_size,
            max_random_scale    = 0.533,  # 480 with imagnet, 32 with cifar10
            min_random_scale    = 0.533,  # 256.0/480.
            data_shape          = (3, 224, 224),
            rand_crop           = False,
            rand_mirror         = False)
            #num_parts           = kv.num_workers,
            #part_index          = kv.rank)
    return train, val

batch_size = 640
data_dir='/efs/datasets/users/furlat/imagenet'
#imagenet
train_imag, val_imag = imagenet_iterator(data_dir,batch_size)



In [ ]:
from importlib import import_module
net = import_module('symbols.resnet_factory')
image_shape = '3,224,224'
label_names = [train_imag.provide_label[0][0]]
num_classes=1000

In [ ]:
#sym = net.get_gated_vmt_symbol(num_classes, 50, image_shape, conv_workspace=256)  
ctx=[mx.gpu(0),mx.gpu(1),mx.gpu(2),mx.gpu(3),mx.gpu(4),mx.gpu(5),mx.gpu(6),mx.gpu(7),mx.gpu(8),mx.gpu(9),mx.gpu(10),mx.gpu(11),mx.gpu(12),mx.gpu(13),mx.gpu(14),mx.gpu(15)]
#ctx=[mx.gpu(0)]
#sym,data,labels=net.get_symbol_mbranch(num_classes=1000,active=1,num_branch=2,gate_prefix='imagenet',rescale_grad=1, num_layers=50,gated=True, image_shape=image_shape)
# sym,data,labels=net.get_symbol(num_classes=1000,active=1,gate_prefix=None,rescale_grad=1, num_layers=50,gated=True, image_shape=image_shape)
sym,data,labels=net.get_symbol_2branch(num_classes=1000,active=1,gate_prefix=None,rescale_grad=1, num_layers=50,gated=True, image_shape=image_shape)

In [ ]:
imagenet_weights= '/efs/datasets/users/furlat/results_imagenet/imagenet_r50-lr05'
model_prefix= 'tryhard-resnet'
prefix = imagenet_weights
#prefix = model_prefix
epoch=120
save_dict = nd.load('%s-%04d.params' % (prefix, epoch))

arg_params_imag = {}
aux_params_imag = {}
for k, v in save_dict.items():
    tp, name = k.split(':', 1)
    if tp == 'arg':
        arg_params_imag[name] = v
    if tp == 'aux':
        aux_params_imag[name] = v
# del arg_params_imag['fc1_bias']
# del arg_params_imag['fc1_weight']
symlist=sym.list_arguments()
gatelist=[s for s in symlist if 'gate' in s]
#print gatelist

In [ ]:
#fixed_param_names=arg_params_imag.keys()+gatelist,
#arg_params_imag.keys()+gatelist
mod = mx.mod.Module(sym, label_names=label_names,fixed_param_names=arg_params_imag.keys(),context=ctx)
#
checkpoint = mx.callback.module_checkpoint(mod,model_prefix)
mod.bind(data_shapes=train_imag.provide_data, label_shapes=val_imag.provide_label)
# mod.init_params()
# print mod.get_params()[0]['bn0_beta'].asnumpy()
#mod.set_params(arg_params_imag, aux_params_imag, allow_missing=True)
mod.init_params(initializer=mx.initializer.Uniform(0.01), arg_params=arg_params_imag, aux_params=aux_params_imag,
                    allow_missing=True, force_init=False)
# print mod.get_params()[0]['bn0_beta'].asnumpy()

In [ ]:
arg_params_imag.keys()+gatelist
mod.get_params()[0]['_a_stage1_unit1_conv3_gate'].asnumpy()

In [ ]:
# #ctx=[mx.gpu(0)]
# mod = mx.mod.Module(sym, label_names=label_names, context=ctx)
# mod.bind(data_shapes=train_imag.provide_data,label_shapes=val_imag.provide_label)
# mod.init_params()
# # # mod.init_params()
# mod.get_params()[0]['stage1_unit1_bn2_beta'].asnumpy()
# mod.get_params()[0]['stage1_unit1_conv3_imagenet_gate'].asnumpy()

In [ ]:
#mx.viz.plot_network(sym, shape={"data":(1,3, 224, 224)})

In [ ]:
model_prefix = '/home/ubuntu/results/resnet-factory-gated' 
#pre_trained_imagenet = '/efs/datasets/users/furlat/results_imagenet/imagenet_r50-lr05'
checkpoint = mx.callback.module_checkpoint(mod,model_prefix)

In [ ]:
mod.fit(train_imag,
        eval_data=val_imag,
        eval_metric=[Cross_Entropy()],
        #eval_metric=[mx.metric.Accuracy()],

        batch_end_callback = [mx.callback.log_train_metric(1),mx.callback.Speedometer(batch_size,1)],
        epoch_end_callback=checkpoint,
        allow_missing=False,
        begin_epoch=0,
        log_prefix = model_prefix,
        optimizer_params={'learning_rate':0.00005, 'momentum': 0.9,'wd':0.0001 },
        num_epoch=25)

In [ ]:
mod.get_params()[0]['_a_stage3_unit1_conv4_gate'].asnumpy()

In [ ]:
mod.get_params()[0]['_a_stage1_unit1_conv1_weight'].asnumpy()
#mod.get_params()[0]['2_stage1_unit1_conv1_weight'].asnumpy()